In [1]:
import os
import pandas as pd

# set tej_key and base
os.environ['TEJAPI_KEY'] = tej_key 
os.environ['TEJAPI_BASE'] = api_base

# set date
start = '2023-01-01'
end = '2023-01-31'    

In [2]:
from zipline.sources.TEJ_Api_Data import get_universe

pool = get_universe(start, end, idx_id = 'IX0027')  # 取得電子類指數代碼

Currently used TEJ API key call quota 532/100000 (0.53%)
Currently used TEJ API key data quota 4251958/10000000 (42.52%)


In [3]:
tickers = ' '.join(pool)

os.environ['mdate'] = start+' '+end
os.environ['ticker'] = tickers

!zipline ingest -b tquant

Merging daily equity files:
Currently used TEJ API key call quota 535/100000 (0.54%)
Currently used TEJ API key data quota 4254327/10000000 (42.54%)


[2024-03-28 09:36:02.557530] INFO: zipline.data.bundles.core: Ingesting tquant.
[2024-03-28 09:36:05.415709] INFO: zipline.data.bundles.core: Ingest tquant successfully.


In [4]:
import TejToolAPI

columns = ['Market', 'ROI', 'PER_TEJ']
data = TejToolAPI.get_history_data(start = start, 
                                   end = end,
                                   ticker=pool, 
                                   columns=columns
                                   )
data = data.sort_values(['coid','mdate'])
data

Currently used TEJ API key call quota 560/100000 (0.56%)
Currently used TEJ API key data quota 4338756/10000000 (43.39%)


,coid,mdate,Market,PER_TEJ,ROI
0,1471,2023-01-03,TWSE,NaN,0.1307
50,1471,2023-01-04,TWSE,NaN,0.0000
100,1471,2023-01-05,TWSE,NaN,-0.1305
150,1471,2023-01-06,TWSE,NaN,0.2614
200,1471,2023-01-09,TWSE,NaN,1.5645
...,...,...,...,...,...
5343,9912,2023-01-13,TWSE,34.2774,4.3386
5359,9912,2023-01-16,TWSE,34.2774,-2.5355
5375,9912,2023-01-17,TWSE,34.2774,-1.4568
5391,9912,2023-01-30,TWSE,34.2774,1.0560


In [5]:
from zipline.data import bundles

bundle_name = 'tquant'
bundle = bundles.load(bundle_name)

In [6]:
from zipline.pipeline.data.dataset import Column, DataSet
from zipline.pipeline.domain import TW_EQUITIES

class OtherDatasets(DataSet):
     
    Market = Column(dtype=object)    
    ROI = Column(dtype=float)  
    PER_TEJ = Column(dtype=float)
 
    domain = TW_EQUITIES

In [23]:
sids = bundle.asset_finder.equities_sids
assets = bundle.asset_finder.retrieve_all(sids)
symbol_mapping_sid = {i.symbol:i.sid for i in assets}
transform_data = data.set_index(['coid', 'mdate']).unstack('coid')
transform_data = transform_data.rename(columns = symbol_mapping_sid)
transform_data.index=transform_data.index.tz_localize('UTC')
transform_data

Market                                            \
coid                         0     1     2     3     4     5     6     7     
mdate                                                                        
2023-01-03 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-04 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-05 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-06 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-09 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-10 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-11 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-12 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-13 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-16 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-17 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-30 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   
2023-01-31 00:00:00+00:00   TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE  TWSE   

                                       ...     ROI                          \
coid                        8     9    ...     406     407     408     409   
mdate                                  ...                                   
2023-01-03 00:00:00+00:00  TWSE  TWSE  ...  0.5291 -0.4902  0.2762 -0.1381   
2023-01-04 00:00:00+00:00  TWSE  TWSE  ...  0.0000  0.0000  2.3416  0.5533   
2023-01-05 00:00:00+00:00  TWSE  TWSE  ...  0.1316  1.9704  0.0000  0.8253   
2023-01-06 00:00:00+00:00  TWSE  TWSE  ...  0.2628 -0.4831  0.4038  0.6821   
2023-01-09 00:00:00+00:00  TWSE  TWSE  ...  0.9174  0.0000  2.1448  0.2710   
2023-01-10 00:00:00+00:00  TWSE  TWSE  ...  1.2987 -0.9709  2.6247  0.1351   
2023-01-11 00:00:00+00:00  TWSE  TWSE  ... -0.2564  0.0000  0.3836 -0.2699   
2023-01-12 00:00:00+00:00  TWSE  TWSE  ... -0.5141 -0.4902  0.3822 -0.1353   
2023-01-13 00:00:00+00:00  TWSE  TWSE  ... -0.5168 -0.4926  1.6497 -0.8130   
2023-01-16 00:00:00+00:00  TWSE  TWSE  ...  0.3896  0.9901  1.2484  0.0000   
2023-01-17 00:00:00+00:00  TWSE  TWSE  ...  0.6468 -0.9804  5.0555  0.1366   
2023-01-30 00:00:00+00:00  TWSE  TWSE  ...  1.0283  2.9703  0.9390  0.9550   
2023-01-31 00:00:00+00:00  TWSE  TWSE  ...  0.2545 -0.9615  0.0000 -0.1351   

                                                                           
coid                          410     411     412     413     414     415  
mdate                                                                      
2023-01-03 00:00:00+00:00  0.4710 -0.7246  0.4808 -0.2415 -0.1002  0.4372  
2023-01-04 00:00:00+00:00  1.7188  1.0949  0.4785 -0.7264  0.2006  0.1088  
2023-01-05 00:00:00+00:00 -1.0753  1.4440 -0.9524  1.7073  0.1001  1.0870  
2023-01-06 00:00:00+00:00  1.8634  0.3559  1.4423  1.4388  1.0000  0.1075  
2023-01-09 00:00:00+00:00 -0.4573  4.0780  1.4218 -0.2364  0.4950  0.2148  
2023-01-10 00:00:00+00:00 -0.4594  1.5332 -0.9346  1.3033  0.0000 -0.3215  
2023-01-11 00:00:00+00:00 -0.6154  3.1879  0.4717  0.1170 -0.4926 -0.7527  
2023-01-12 00:00:00+00:00 -0.6192 -3.4146  0.4695 -0.9346 -0.4950  2.3835  
2023-01-13 00:00:00+00:00 -0.9346 -4.5455 -2.8037 -0.7075  0.4975  4.3386  
2023-01-16 00:00:00+00:00  0.9434 -1.2346  0.4808 -0.2375  0.0000 -2.5355  
2023-01-17 00:00:00+00:00  0.0000  0.3571  0.4785  0.1190  0.0000 -1.4568  
2023-01-30 00:00:00+00:00  1.7134  2.8470  5.7143  1.4269  0.4950  1.0560  
2023-01-31 00:00:00+00:00  3.6753 -0.5190  2.7027  1.7585  2.4631  0.0000  

[13 rows x 1248 columns]

In [16]:
from zipline.pipeline.loaders.frame import DataFrameLoader
custom_loader = {}

inputs=[
        OtherDatasets.Market,
        OtherDatasets.ROI,
        OtherDatasets.PER_TEJ,  
        ]

for i in inputs:
    custom_loader[i]=DataFrameLoader(column=i,
                                     baseline=transform_data[i.name])

custom_loader

{OtherDatasets<TW>.Market::object: <zipline.pipeline.loaders.frame.DataFrameLoader at 0x1639b43a810>,
 OtherDatasets<TW>.ROI::float64: <zipline.pipeline.loaders.frame.DataFrameLoader at 0x1639b43bd10>,
 OtherDatasets<TW>.PER_TEJ::float64: <zipline.pipeline.loaders.frame.DataFrameLoader at 0x1639b43a110>}

In [17]:
from zipline.pipeline import SimplePipelineEngine
from zipline.pipeline.data import TWEquityPricing
from zipline.pipeline.loaders import EquityPricingLoader
pricing_loader = EquityPricingLoader.without_fx(bundle.equity_daily_bar_reader,
                                                bundle.adjustment_reader)
def choose_loader(column):
    if column.name in TWEquityPricing._column_names:
        return pricing_loader
    elif column.name in OtherDatasets._column_names:     
        return custom_loader[column]
    else:
        raise Exception('Column not available')
    
engine = SimplePipelineEngine(get_loader = choose_loader,
                              asset_finder = bundle.asset_finder,
                              default_domain = TW_EQUITIES)

In [18]:
from zipline.pipeline import Pipeline

def make_pipeline():

    return Pipeline(
        columns={
            'price': TWEquityPricing.close.latest,
            'Market': OtherDatasets.Market.latest,
            'ROI': OtherDatasets.ROI.latest,
            'PER_TEJ': OtherDatasets.PER_TEJ.latest
            }
    )

start_dt = pd.Timestamp(start, tz = 'UTC')
end_dt = pd.Timestamp(end, tz = 'UTC')

pipeline_result = engine.run_pipeline(make_pipeline(), start_dt, end_dt)
pipeline_result 

price Market  ROI  PER_TEJ
2023-01-04 00:00:00+00:00 Equity(0 [1471])      7.66    N/A  NaN      NaN
                          Equity(1 [1582])     59.10    N/A  NaN      NaN
                          Equity(2 [2059])    382.50    N/A  NaN      NaN
                          Equity(3 [2301])     64.20    N/A  NaN      NaN
                          Equity(4 [2302])     16.75    N/A  NaN      NaN
...                                              ...    ...  ...      ...
2023-01-31 00:00:00+00:00 Equity(411 [8249])   28.90    N/A  NaN      NaN
                          Equity(412 [8261])  111.00    N/A  NaN      NaN
                          Equity(413 [8271])   42.65    N/A  NaN      NaN
                          Equity(414 [8499])  101.50    N/A  NaN      NaN
                          Equity(415 [9912])    9.57    N/A  NaN      NaN

[4992 rows x 4 columns]